In [1]:
import math
from tletools import TLE

DAY = 24*60*60
EARTH_RADIUS = 6371000.0
EARTH_MU = 3.986004418 * 10**14
EARTH_ROTATION = 7.2921158553 * 10**-5
DEG_TO_RAD = math.pi/180
RAD_TO_DEG = 180/math.pi

def lat_lon(r, x, y, z):
    lat = math.asin(z / r) * RAD_TO_DEG

    if x > 0:
        lon = math.atan(y/x) * RAD_TO_DEG
    elif y > 0:
        lon = math.atan(y/x) * RAD_TO_DEG + 180
    else:
        lon = math.atan(y/x) * RAD_TO_DEG - 180

    return lat, lon

def find_eccentric_anomaly(M, e):
    E = M

    if e > 0.5:
        E = math.pi*2


    while abs(M-(E-e*math.sin(E))) > 0.00001:
        E = E - (E-(M+e*math.sin(E)))/(1-e*math.cos(E))

    return E

tle_string = """
STARLINK BK             
1 45787U 20038BK  20171.16668981  .00133620  11278-4  16465-3 0  9993
2 45787  52.9999 269.6622 0111117  71.1170  21.0884 15.96235573  2010
"""

tle_string = """
STARLINK-1362           
1 45538U 20025H   20171.16668981 -.00012324  00000-0 -82793-3 0  9996
2 45538  52.9976 126.8573 0001595  80.3082 190.2938 15.05587840  9574
"""

tle_lines = tle_string.strip().splitlines()
tle = TLE.from_lines(*tle_lines)

print(tle)

name=tle.name
norad=tle.norad
designator=tle.int_desig
epoch = tle.epoch_day
mean_motion = tle.n
eccentricity = tle.ecc
mean_anomaly = tle.M
inclination = tle.inc
ascension = tle.raan
argument_of_perigee = tle.argp

print(name,norad,designator)
print("Epoch: %f" % epoch)
print("Mean motion: %f orbits/day" % mean_motion)
print("Eccentricity: %f" % eccentricity)
print("Mean anomaly: %f degrees" % mean_anomaly)
print("Inclination: %f degrees" % inclination)
print("Ascension %f degrees" % ascension)
print("Perigee: %f degrees" % argument_of_perigee)

print ("----")
orbital_period = DAY / mean_motion
print("Orbital period: %f minutes" % (orbital_period / 60))



TLE(name='STARLINK-1362', norad='45538', classification='U', int_desig='20025H', epoch_year=2020, epoch_day=171.16668981, dn_o2=-0.00012324, ddn_o6=0.0, bstar=-0.00082793, set_num=999, inc=52.9976, raan=126.8573, ecc=0.0001595, argp=80.3082, M=190.2938, n=15.0558784, rev_num=957)
STARLINK-1362 45538 20025H
Epoch: 171.166690
Mean motion: 15.055878 orbits/day
Eccentricity: 0.000160
Mean anomaly: 190.293800 degrees
Inclination: 52.997600 degrees
Ascension 126.857300 degrees
Perigee: 80.308200 degrees
----
Orbital period: 95.643706 minutes


In [2]:

semi_major_axis = (((orbital_period / (math.pi*2))**2) * EARTH_MU) ** (1/3)
print("Semi-major axis: %f km" % (semi_major_axis / 1000.0))

mean_motion_rad_sec = math.sqrt(1 / ((orbital_period / (math.pi*2))**2))
print("Mean motion: %f rad/sec" % mean_motion)

last_perigee = mean_anomaly*DEG_TO_RAD / mean_motion_rad_sec
print("Last perigee: %f sec before epoch" % last_perigee)

eccentric_anomaly = find_eccentric_anomaly(mean_anomaly * DEG_TO_RAD, eccentricity) * RAD_TO_DEG
print("Eccentric anomaly: %f degrees" % eccentric_anomaly)

true_anomaly = 2*math.atan2(math.sqrt(1 + eccentricity)*math.sin(eccentric_anomaly*DEG_TO_RAD/2), math.sqrt(1 - eccentricity)*math.cos(eccentric_anomaly*DEG_TO_RAD/2)) * RAD_TO_DEG
print("True anomaly: %f degrees" % true_anomaly)

radius = semi_major_axis*(1-eccentricity*math.cos(eccentric_anomaly*DEG_TO_RAD))
print("Radius: %f m" % (radius / 1000.0))

angular_momentum = math.sqrt(EARTH_MU * semi_major_axis * (1 - eccentricity**2))
print("Angular momentum: %f m/s" % angular_momentum)

cos_arg_tru, sin_arg_tru = math.cos(argument_of_perigee*DEG_TO_RAD + true_anomaly*DEG_TO_RAD), \
     math.sin(argument_of_perigee*DEG_TO_RAD + true_anomaly*DEG_TO_RAD)

cos_inc, sin_inc = math.cos(inclination*DEG_TO_RAD), math.sin(inclination*DEG_TO_RAD)
cos_asc, sin_asc = math.cos(ascension*DEG_TO_RAD), math.sin(ascension*DEG_TO_RAD)
cos_tru, sin_tru = math.cos(true_anomaly*DEG_TO_RAD), math.sin(true_anomaly*DEG_TO_RAD)

coords = (
    radius*(cos_asc*cos_arg_tru - sin_asc*sin_arg_tru*sin_inc),
    radius*(sin_asc*cos_arg_tru - cos_asc*sin_arg_tru*sin_inc),
    radius*(sin_inc*sin_arg_tru),
)

velocity = (
    ((coords[0]*angular_momentum*eccentricity) / (radius*orbital_period))*sin_tru - \
        (angular_momentum / radius)*(cos_asc*sin_arg_tru + sin_asc*cos_arg_tru*cos_inc),
    ((coords[1]*angular_momentum*eccentricity) / (radius*orbital_period))*sin_tru - \
        (angular_momentum / radius)*(sin_asc*sin_arg_tru + cos_asc*cos_arg_tru*cos_inc),
    ((coords[2]*angular_momentum*eccentricity) / (radius*orbital_period))*sin_tru - \
        (angular_momentum / radius)*(sin_inc*cos_arg_tru),

)

a = velocity[0] + EARTH_ROTATION*coords[1]
b = velocity[1] - EARTH_ROTATION*coords[0]

theta = epoch * math.pi * 2
cos_theta, sin_theta = math.cos(theta), math.sin(theta)
x = a*cos_theta + b*sin_theta
y = a*-sin_theta + b*cos_theta
z = velocity[2]

ap_plus_pe = semi_major_axis * 2
ap_minus_pe = eccentricity * ap_plus_pe
apogee = (ap_plus_pe + ap_minus_pe) / 2
perigee = apogee - ap_minus_pe

#calculated_eccentricity = (apogee-perigee) / (apogee+perigee)

#assert (apogee > 0) and (perigee > 0) and (apogee > perigee)
#assert abs(eccentricity - calculated_eccentricity) < 0.00001

apogee -= EARTH_RADIUS
perigee -= EARTH_RADIUS

print("Apogee: %f km" % (apogee / 1000.0))
print("Perigee: %f km" % (perigee / 1000.0))

lat, lon = lat_lon(radius, x, y, z)
print("Coordinates at epoch: %f, %f" % (lat, lon))

Semi-major axis: 6927.838830 km
Mean motion: 15.055878 rad/sec
Last perigee: 3033.400695 sec before epoch
Eccentric anomaly: 190.292167 degrees
True anomaly: 190.290535 degrees
Radius: 6928.926041 m
Angular momentum: 52549401026.079613 m/s
Apogee: 557.943821 km
Perigee: 555.733840 km
Coordinates at epoch: 0.001200, 70.216050
